In [7]:
import sklearn
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [10]:
class RandomForest():
    """A class to choose and train a random forest model"""
    """This implements the sklearn Random Forest Model"""
    
    def __init__(self, data_dir='./processed_data/', regressor=False, n_jobs=1):
        """Initializes the random forest class"""
        
        # Create an initialization function that will allow for
        # a saved model to be loaded
        
        self.n_jobs = n_jobs
        self.regressor = regressor
        self.data_dir = data_dir
        
        if regressor:
            self.model = RandomForestRegressor()
        else:
            self.model = RandomForestClassifier()
            
        self.params = {}
        self.features = []
        
    def read_data(self):
        """Read in the data from the specified directory"""
        
        self.cv_X_df = pd.read_csv(self.data_dir+'cv_X.csv')
        self.cv_y_df = pd.read_csv(self.data_dir+'cv_y.csv')
        self.cv_ids_df = pd.read_csv(self.data_dir+'cv_ids.csv')
        self.train_X_df = pd.read_csv(self.data_dir+'train_X.csv')
        self.train_y_df = pd.read_csv(self.data_dir+'train_y.csv')
        self.train_ids_df = pd.read_csv(self.data_dir+'train_ids.csv')
        self.test_X_df = pd.read_csv(self.data_dir+'test_X.csv')
        self.test_ids_df = pd.read_csv(self.data_dir+'test_ids.csv')
        
    def select_features(self):
        """Select features from the dataset by using feature importances
        to add features one by one for selection"""
        
        # TODO:
        #   - PERFORM FEATURE SELECTION
        
        self.selected_features = self.features
        
    def tune_params(self):
        """Function to handle tuning hyperparameters of the model"""
        
        # TODO:
        #   - PERFORM HYPERPARAMETER SEARCH HERE
        self.params = self.params
        
    def train_model(self):
        """Trains the model on all training data
        and returns a model to be used for prediction"""
        self.model.fit(self.train_X_df.values, self.train_y_df.values[:, 1])
        
    def predict_output(self):
        """Make predictions on test data."""
        pred = self.model.predict(self.test_X_df.values)
        return pred
        
    def get_model(self):
        """Return the model used for prediction"""
        
        # TODO:
        #    - HOW TO RETURN THIS?? JUST RETURN THE SKLEAN MODEL???
        
        return self.model
        

In [11]:
model = RandomForest()

model.read_data()

model.select_features()

model.tune_params()

model.train_model()

pred = model.predict_output()
print(pred)

train_model = model.get_model()

[1 1 1 1]
